#### Required imports for analysis part

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
#check all mdp files
# 1. number of steps
# 2. columb type
# 3. temperature coupling groups

#### Create ligand pdb

In [1]:
# separates ligand from protein ligand complex and create ligand.pdb file
# change the ligand name wherever needed
ligand = "JZ4"
! grep $ligand 3HTB.pdb > ligand.pdb

grep: 3HTB.pdb: No such file or directory


#### Remove waters and non-standard residues

In [ ]:
# pdb2gmx takes only protein file containing standard amino acid residues,
    # so check the protein file, should contain only standard residues
! grep -v HETATM protein.pdb > protein_processed.pdb

#### 1. Protein topology

In [ ]:
# this command prepares protein topology file and sets water model
#system topology topol.top, a position restraint file posre.itp (included in the topology file), and a coordinate file.gro
! printf "1" | gmx pdb2gmx -f protein_processed.pdb -o protein_processed.gro -water spc

In [ ]:
! tail topol.top

#### 2. Ligand topology

In [ ]:
# for ligand toplogy
# 1. convert to mol2 file
# 2. Check the ligand variable in 1st step
# 3. change number before ligand column to "1" and ligand name in mol2 file
# 4. change ligand residue name in mol2 file
# --tile option is used to specify name of ligand in 2nd row 
! obabel ligand.pdb --title JZ4 -O ligand.mol2 -h

In [ ]:
# 4. fix bond order with perl script
! perl scripts/sort_mol2_bonds.pl ligand.mol2 ligand_fix.mol2

In [ ]:

# 5. goto cgenFF website and upload the previous mol2 file and get the corresponding .str file. 
    #(cgenFF website is used only if CHARMM36 Forcefield is used)
    #The CHARMM stream file is the ligand topology - atom types,charges, and bonded connectivity.
# Examine the contents of .str and look at the penalties for the charges and the new dihedral parameters. 
    #If all of them are very low, this topology is of very good quality 
        #and can be used directly for our simulation.

# 6. Use the cgenff_charmm2gmx.py script that downloaded from the MacKerell website.    
        #this command creates jz4_ini.pdb ligand structure file, ligand.prm dihedral parameters
# Change ligand id in below command - important

! python3 scripts/cgenff_charmm2gmx_py3_nx2.py JZ4 ligand_fix.mol2 ligand_fix.str charmm36-feb2021.ff/